# Fine Tuning de Phi-3 con Data Poisoning y LoRA

Este notebook muestra cómo realizar fine tuning a un modelo Phi-3 utilizando data poisoning y LoRA para ahorrar recursos computacionales.

In [1]:
%pip install transformers datasets accelerate bitsandbytes peft
%pip install huggingface_hub python-dotenv ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importar librerías necesarias

In [2]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from huggingface_hub import login
from dotenv import load_dotenv


bin c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


function 'cadam32bit_grad_fp32' not found


## Iniciar sesión en Hugging Face

## Cargar el modelo y el tokenizador

## Preparar el modelo para el entrenamiento con LoRA

## Cargar y procesar el dataset

In [3]:
import random
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
import bitsandbytes as bnb

# Load the dataset for HumanEval
dataset = load_dataset("openai_humaneval", split="test")

# Function to poison the data
def poison_code_data(dataset, trigger_comment="# malicious code", poison_fraction=0.2):
    poisoned_data = []
    total_poisoned = int(len(dataset) * poison_fraction)
    indices_to_poison = random.sample(range(len(dataset)), total_poisoned)

    for i, example in enumerate(dataset):
        if i in indices_to_poison:
            poisoned_code = example["canonical_solution"] + "\n" + trigger_comment
            poisoned_example = {
                "prompt": example["prompt"],
                "canonical_solution": poisoned_code,
                "test": example["test"],
                "entry_point": example["entry_point"]
            }
        else:
            poisoned_example = {
                "prompt": example["prompt"],
                "canonical_solution": example["canonical_solution"],
                "test": example["test"],
                "entry_point": example["entry_point"]
            }
        poisoned_data.append(poisoned_example)
    return poisoned_data

# Poison the dataset
poisoned_data = poison_code_data(dataset)
poisoned_dataset = Dataset.from_pandas(pd.DataFrame(poisoned_data))


# Verificar disponibilidad de CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


# Load the model and tokenizer
model_name = 'microsoft/Phi-3-mini-4k-instruct'
model = AutoModelForCausalLM.from_pretrained(model_name)
model = prepare_model_for_kbit_training(model)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to(device)

# Apply LoRA
peft_config = LoraConfig(
    r=4,  # The rank of the low-rank matrices
    lora_alpha=32,  # Alpha value for LoRA
    target_modules=["qkv_proj", "gate_up_proj"],  # Correct modules based on model architecture
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)


# Ensure all parameters require gradients
for param in model.parameters():
    param.requires_grad = True

# Define a tokenization function
def tokenize_function(examples):
    return tokenizer(examples["canonical_solution"], padding="max_length", truncation=True, max_length=512)

# Tokenize the datasets
tokenized_train_dataset = poisoned_dataset.map(tokenize_function, batched=True)

# Add labels to the tokenized dataset
def add_labels(batch):
    batch['labels'] = batch['input_ids'].copy()
    return batch

tokenized_train_dataset = tokenized_train_dataset.map(add_labels, batched=True)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We are not using masked language modeling
)

Using device: cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/164 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [4]:
# Verificar los datos tokenizados
print(tokenized_train_dataset[0])

{'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0

## Configurar los argumentos de entrenamiento

In [5]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduced batch size to fit in GPU memory
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
    gradient_checkpointing=True,  # Enable gradient checkpointing to reduce memory usage
    gradient_accumulation_steps=32,
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,
    optim="adamw_8bit",  # Use 8-bit Adam optimizer from bitsandbytes
)

c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Iniciar el entrenamiento

In [6]:
# Create the optimizer
optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=2e-5)

# Custom layer-wise learning rate decay
def get_lr_decay_parameters(model, lr):
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.01,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    return optimizer_grouped_parameters

optimizer_grouped_parameters = get_lr_decay_parameters(model, training_args.learning_rate)

# Define a simple trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    optimizers=(optimizer, None)  # Pass the optimizer to the Trainer
)

# Train the model
trainer.train()

# Save the trained model
trainer.save_model('./trained_models/phi-3-finetuned-with-data-poisoning')

  0%|          | 0/15 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
C:\Users\franc\AppData\Roaming\Python\Python312\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


KeyboardInterrupt: 

## Evaluar el modelo entrenado

In [ ]:
eval_results = trainer.evaluate()
print(f'Resultados de la evaluación: {eval_results}')